In [10]:
# imports

import  time
import numpy as np
import pandas as pd
# import sys
# for FieldMax powermeter dll wrapper
from pyFieldMaxII.fieldmax import fieldmax
#import visa for device connection
# import pyvisa
# import pyvisa_py
# import usb
# import usb.core
# import usb.util
# import pathlib
# import os
import libximc.highlevel as ximc


In [11]:

#-------FieldMaxII connection--------------
path_to_fm2 = r'C:\Program Files (x86)\Coherent\FieldMaxII PC\Drivers\Win10\FieldMax2Lib\x64\FieldMax2Lib.dll'
fm2 = fieldmax.FieldMax(path_to_fm2)
fm2.openDriver()
print(fm2.get_SerialNumber())
fm2.sync()
print('FM2 Connected and initial reading: ', fm2.get_dataPoint())


# connecting rotor
# Devices search
devices = ximc.enumerate_devices(
    ximc.EnumerateFlags.ENUMERATE_NETWORK |
    ximc.EnumerateFlags.ENUMERATE_PROBE
)

if len(devices) == 0:
    print("The real devices were not found.")
else:
    # Print real devices list
    print("Found {} real device(s):".format(len(devices)))
    for device in devices:
        print("  {}".format(device))

# device_uri = "xi-emu:///{}".format(virtual_device_file_path)
device_uri = r"xi-com:\\.\COM5" #for usb in our case
# device_uri = r"xi-com:///dev/ttyACM29"
# device_uri = "xi-tcp://172.16.131.140:1820"
# device_uri = "xi-net://192.168.1.120/abcd"

# making a rotor object 
rotor = ximc.Axis(device_uri)
# To open the connection, you must manually call `open_device()` method

#--------------
#zeroing of motor

rotor.open_device()
print('current position', rotor.get_position())
print('moving rotor to reference zero', rotor.command_move(0,0))
rotor.close_device()


1126C18R
FM2 Connected and initial reading:  -0.03519144281744957
Found 2 real device(s):
  {'uri': 'xi-com:\\\\.\\COM3', 'device_serial': 12630, 'Manufacturer': 'XIMC', 'ManufacturerId': 'SM', 'ProductDescription': 'XISM-USB', 'Major': 2, 'Minor': 3, 'Release': 1, 'ControllerName': '', 'CtrlFlags': 1, 'PositionerName': '257nm'}
  {'uri': 'xi-com:\\\\.\\COM5', 'device_serial': 13160, 'Manufacturer': 'XIMC', 'ManufacturerId': 'SM', 'ProductDescription': 'XISM-USB', 'Major': 2, 'Minor': 3, 'Release': 1, 'ControllerName': 'Axis 2', 'CtrlFlags': 0, 'PositionerName': ''}
current position Position: 0
uPosition: 0
EncPosition: 0

moving rotor to reference zero None


In [12]:
N = 20*8


In [13]:


#external csv where all is written; for each Laser Intensity 1 csv file
export_data = pd.DataFrame(columns= [
    'rotor_step',
    'raw_data',
    'averaged',
    'stdev'
])

In [14]:

# function for measuring at a certain power meter type, flag is for noting down which steps need to be measured with th100 again
def measure(i=0, N = N, measurements = np.zeros((int(N/8), 8), dtype=float)):
    while N > 0:
        meas = np.array(fm2.get_dataArray())
        flag  = (np.any(meas == 0.0))
        print(flag, meas)
        looping = 0
        while flag:
            print('zero in', i, meas)
            if looping > 10:
                time.sleep(19)
            meas = np.array(fm2.get_dataArray())
            flag  = np.any(meas == 0.0)
            if flag== False:
                print('dobre', meas)
            looping += 1
        measurements[i] = meas
        i += 1
        N -= len(meas)
        print(N)
        time.sleep(0.1)

    print(f'{100-N} datapoints measured on {fm2}')
        
    return measurements #np array floats length N

In [15]:
rotor.open_device()
#Measurement process

steps = np.arange(0, 36000
                  , 200) #defined rotor steps

# rotor.open_device()
for i, step in enumerate(steps):
    time.sleep(1)
    step  = int(step)
    rotor.command_move(step, 0)
    print('measuring, rotor moved to ', rotor.get_position())
    raw_data = measure()
    raw_data = np.array(raw_data.flatten(), dtype = float)
    print('PRINTING SHAPE SHOULD BE FLAT', raw_data.shape)
    # print(raw_data)
    export_data.loc[i, 'rotor_step'] = step
    export_data.at[i, 'raw_data'] = raw_data
    export_data.loc[i, 'averaged'] = np.average(raw_data)
    export_data.loc[i, 'stdev'] = np.std(raw_data)
    print(f'average power at step {step} is {np.average(raw_data)} with stdev {np.std(raw_data)}')
    if input('continue?') == 'n':
        break
    


#exporting the csv with the naming acc. to laser intensity
laser_intensity = input('What is the laser setting: ')
export_data.to_csv(f'measurements/measurements_after_HWP_fm2/measured_rf{laser_intensity}.csv', index=False)

#close drivers
fm2.closeDriver()
rotor.close_device()

measuring, rotor moved to  Position: 0
uPosition: 0
EncPosition: 0

False [-0.03359183 -0.03479154 -0.03469156 -0.03559135 -0.03549137 -0.03489152
 -0.03469156 -0.03409171]
152
False [-0.03549137 -0.03479154 -0.03409171 -0.03389176 -0.03469156 -0.03409171
 -0.03389176 -0.03369181]
144
False [-0.03439164 -0.03439164 -0.03439164 -0.03339188 -0.03449161 -0.03439164
 -0.03439164 -0.03499149]
136
False [-0.03549137 -0.03569132 -0.03549137 -0.03529142 -0.03609122 -0.03399174
 -0.03349186 -0.03409171]
128
False [-0.03329191 -0.03439164 -0.03449161 -0.03449161 -0.03539139 -0.03559135
 -0.03459159 -0.03459159]
120
False [-0.03539139 -0.03409171 -0.03449161 -0.03519144 -0.03539139 -0.03459159
 -0.03529142 -0.03469156]
112
False [-0.03479154 -0.03489152 -0.03389176 -0.03369181 -0.03329191 -0.03379178
 -0.03519144 -0.03509147]
104
False [-0.03559135 -0.03469156 -0.03469156 -0.03569132 -0.03549137 -0.03469156
 -0.03389176 -0.03459159]
96
True [-0.03409171 -0.03359183 -0.03379178 -0.03439164 -0.0343